In [1]:
# Importamos las librerías necesarias para convertir el pdf a texto
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

# Definimos la función que convierte el pdf a texto
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

# Guardamos el texto en un nuevo archivo
with open('Parques_General.txt', 'w', encoding='utf-8') as f:
    text = convert_pdf_to_txt('Guia_Parques_Naturales.pdf')
    f.write(text)

In [74]:
# Leemos el texto por líneas
with open('Parques_General.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [63]:
lines = [line for line in lines if line.isdigit() == False and line.startswith('') == False and len(line) != 2 and line.startswith('(') == False]

In [66]:
# Comunidades
com = {'Andalucía':['ALMERÍA', 'CÁDIZ', 'CÓRDOBA', 'GRANADA', 'HUELVA', 'JAÉN', 'MÁLAGA', 'SEVILLA'],
        'Aragón':['HUESCA', 'ZARAGOZA', 'TERUEL'],
        'Principado de Asturias':['PRINCIPADO DE ASTURIAS'],
        'Islas Baleares':['IBIZA', 'MALLORCA', 'FORMENTERA'],
        'Islas Canarias':['LANZAROTE', 'TENERIFE'],
        'Cantabria':['CANTABRIA'],
        'Castilla La Mancha':['GUADALAJARA', 'TOLEDO', 'CIUDAD REAL', 'CUENCA'],
        'Castilla y León':['ÁVILA', 'BURGOS', 'SEGOVIA', 'SORIA', 'PALENCIA', 'VALLADOLID', 'ZAMORA'],
        'Cataluña':['LÉRIDA', 'GERONA', 'TARRAGONA'],
        'Ceuta':['CEUTA'],
        'Comunidad Valenciana':['ALICANTE', 'CASTELLÓN', 'VALENCIA'],
        'Extremadura':['CÁCERES', 'BADAJOZ'],
        'Galicia':['ORENSE', 'LUGO','PONTEVEDRA'],
        'La Rioja':['LA RIOJA'],
        'Comunidad de Madrid':['MADRID'],
        'Región de Murcia':['REGIÓN DE MURCIA'],
        'Navarra': ['NAVARRA'],
        'País Vasco':['ÁLAVA', 'GUIPÚZCOA', 'VIZCAYA'],
        }

# Quitamos saltos de línea
#lines = [line.replace('\n', '') for line in lines]

# Quitamos otras líneas innecesarias
#lines = [line for line in lines if line.isdigit() == False and line.startswith('') == False and len(line) != 2 and line.startswith('(') == False]

# Extraemos los nombres de los centros
centros = {}
for comunidad in com.keys():
    centros[comunidad] = {}
    for provincia in com[comunidad]:
        centros[comunidad][provincia] = []
        for i, line in enumerate(lines):
            if line.startswith(provincia):
                for j in range(i+2, len(lines)-1):
                    if lines[j] == '':
                        break
                    else:
                        centros[comunidad][provincia].append(lines[j])

In [75]:
len(lines)

62423

In [64]:
with open('prueba.txt', 'w', encoding='utf-8') as f:
    f.writelines(lines)

In [5]:
andalucia = centros['Andalucía']

In [6]:
andalucia['ALMERÍA']

['Nautarum Centro de Interpretación',
 'de la pesca',
 'Paraje Natural del Karst en Yesos',
 'de Sorbas. Centro de Visitantes Los',
 'Yesares',
 'Parque Natural Cabo de Gata-Níjar.',
 'Jardín botánico El Albardinal']

In [25]:
anda = []
for prov in andalucia:
    anda.append(' '.join(andalucia[prov]).replace('- ', ''))

In [26]:
anda[0]

'Nautarum Centro de Interpretación de la pesca Paraje Natural del Karst en Yesos de Sorbas. Centro de Visitantes Los Yesares Parque Natural Cabo de Gata-Níjar. Jardín botánico El Albardinal'

In [7]:
import pandas as pd

def get_general_data(centros, lines, provincia):
    # Obtenemos datos de cada centro
    data1 = {'Ubicacion':[], 'Centro':[], 'Direccion':[], 'Telefono':[], 'email':[],'Informacion':[]}

    clean_lines = [line.replace('\n', '') for line in lines]

    for centro in centros:

        for i, line in enumerate(clean_lines[:-5]):

            if centro in line and 'Tel' in ''.join(clean_lines[i+1:i+5]):
                for j in range(i+2, len(clean_lines)):

                    if len(data1['Informacion']) == len(data1['Centro']):
                        break
                    elif clean_lines[j].startswith('Información General'):
                        texto = ''
                        for k in range(j+2, len(clean_lines)):
                            if clean_lines[k] == '':
                                data1['Informacion'].append(texto)
                                break
                            else:
                                texto += ' ' + clean_lines[k]
                    if j == i + 2:
                        # Distintas opciones en función del tipo de dirección
                        if clean_lines[j+1].startswith('Tel'):
                            data1['Direccion'].append(clean_lines[j])
                            data1['Telefono'].append(clean_lines[j+1])
                            data1['email'].append(clean_lines[j+2])

                        elif clean_lines[j+2].startswith('Tel'):
                            data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+1])
                            data1['Telefono'].append(clean_lines[j+2])
                            data1['email'].append(clean_lines[j+3])

                        else:
                            data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+2])
                            data1['Telefono'].append(clean_lines[j+3])
                            data1['email'].append(clean_lines[j+4])

    centros_df = pd.DataFrame(data1, index=range(len(data1['Centro'])))

    return centros_df

In [ ]:
df = 

In [10]:
'Tel' in ''.join(['0', '1', '2', 'Telefono'])

True

In [82]:
Prueba = {'Ubicacion':[], 'Centro':[], 'Direccion':[], 'Telefono':[], 'email':[], 'web':[]}

#lines = [line.replace('\n', '') for line in lines]

for i, line in enumerate(lines[10:-10]):
    if line.startswith('Teléfono'):
        direccion = ''
        telf = line
        email = 'NP'
        web = 'NP'
        for bc in range(i, i-10, -1):
            if lines[bc] == '\n':
                Prueba['Direccion'].append(direccion)
                Prueba['Centro'].append(lines[bc-1])
                if lines[bc-2] != '\n':
                    Prueba['Ubicacion'].append(lines[bc-2])
                else:
                    Prueba['Ubicacion'].append('NP')
                break
            else:
                direccion = lines[bc] + ' ' + direccion
        for ad in range(i+1, i+10):
            if lines[ad].startswith('Email'):
                email = lines[ad]
            elif '@' in lines[ad]:
                email += ' ' + lines[ad]
            elif lines[ad].startswith('Web'):
                web = lines[ad]
            else:
                telf += lines[ad]
        Prueba['Telefono'].append(telf)
        Prueba['email'].append(email)
        Prueba['web'].append(web)

In [83]:
df = pd.DataFrame(Prueba, index=range(len(Prueba['Centro'])))

ValueError: could not broadcast input array from shape (162) into shape (156)

In [84]:
for key, val in Prueba.items():
    try:
        print(key, len(val))
    except:
        print(key, val)

Ubicacion 156
Centro 156
Direccion 156
Telefono 162
email 162
web 162


In [85]:
Prueba['Ubicacion'][:10]

['NP', 'NP', 'NP', 'á', 'NP', 'NP', 'NP', 'n', 'NP', 'NP']

In [86]:
Prueba['Centro'][:10]

['Espacios y Actividades en la Naturaleza Accesibles para Todas las Personas',
 '1188',
 '2222',
 'C',
 '3333',
 'Señalización ',
 '3399',
 'A',
 '4488',
 '5522']

In [87]:
Prueba['Telefono'][:10]

['Teléfono/Fax: 950 139 554\x0cGuía_Completa  22/8/11  14:22  Página 15Centros de interpretación / Casas del Parque y Aulas de la Naturaleza“Nautarum”- Centro de Interpretración de la PescaCarretera Garrucha - Mojácar Playa 04630 Garrucha (Almería)',
 'Teléfono: 950 364 563 \x0cGuía_Completa  22/8/11  14:22  Página 19Centros de interpretación / Casas del Parque y Aulas de la NaturalezaParaje Natural del Karst en Yesos de SorbasCentro de Visitantes “Los Yesares”C/ Terraplén, s/n04270 Sorbas (Almería)',
 'Teléfono: 671 561 226\x0cGuía_Completa  22/8/11  14:23  Página 23Centros de interpretación / Casas del Parque y Aulas de la NaturalezaParque Natural Cabo de Gata-NíjarJardín Botánico El AlbardinalC/ Fundición s/n04071 Rodalquilar – Níjar (Almería)',
 'Teléfono: 956 243 474dnA2288C/ Buen Pastor, 507 (Ctra. de Camposoto, juntoacceso a la playa)11100 San Fernando (Cádiz)',
 'Teléfono: 956 106 796  Fax: 671 564 113\x0cGuía_Completa  22/8/11  14:23  Página 34Espacios y Actividades en la Natu

In [39]:
df.loc[0, 'Telefono']

'Teléfono/Fax: 950 139 554Centros de interpretación / Casas del Parque y Aulas de la Naturaleza“Nautarum”- Centro de Interpretración de la PescaCarretera Garrucha - Mojácar Playa 04630 Garrucha (Almería)'

In [92]:
from ghostscript import Ghostscript

In [89]:
import ctypes
from ctypes.util import find_library
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

'C:\\Program Files\\gs\\gs9.54.0\\bin\\gsdll64.dll'